# Магические методы

Магический метод, это метод определенный в классе, который начинается и заканчивается с двух подчеркиваний:  
https://docs.python.org/3/reference/datamodel.html

## Методы `__new__`, `__init__`, `__str__`, `__hash__`, `__eq__`

In [1]:
class User():

    # Конструктор класса. Метод переопределяет момент создания объекта класса
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls) # Возвращаем созданный экземпляр класса

    # Метод переопределяет инициализацию объекта
    def __init__(self, name, email):
        self.name = name
        self.email = email

    # Метод переопределяет строковое представление класса
    def __str__(self):
        return '{} <{}>'.format(self.name, self.email)

    # Метод переопределяет функцию хеширования которая используется, например,
    # когда мы получаем ключи в словаре.
    def __hash__(self):
        return hash(self.email)

    # Метод переопределяет и реализует оператор == (equals)
    def __eq__(self, obj):
        # Сравниваем по атрибуту email, если они равны, то считаем, что и объекты в целом равны:
        return self.email == obj.email



jane = User('Jane Doe', 'janedoe@example.com')
joe = User('Joe Doe', 'janedoe@example.com')

Переопределен оператор `==` или магический метод `__eq__()`, поэтому мы можем сравнивать объекты:

In [2]:
print('jane == joe?', jane == joe)

jane == joe? True


Оба объекта имеют одинаковый хеш, потому что он вычисляется только по атрибуту `email`:

In [3]:
print('hash(jane) =', hash(jane))
print('hash(joe)  =', hash(joe))

hash(jane) = 4589294839918257202
hash(joe)  = 4589294839918257202


Попробуем добавить объекты `jane` и `joe` в обычный словарь:

In [4]:
{user.__class__: user.name for user in [jane, joe]}

{__main__.User: 'Joe Doe'}

В словаре только один _последний_ добавленный объект! Потому, что в словаре элементы хранятся по уникальным хешам, а объекты `jane` и `joe` имеют одинаковый хеш!

## Метод `__new__` и шаблон `singleton`

Магический метод `__new__` отвечает за момент создания объекта класса.

Создадим класс который будет всегда возвращать ссылку на единственный экземпляр класса. Переменные `a` и `b` будут ссылаться на один и тот же экземпляр класса:

In [5]:
class Singleton():
    instance = None

    def __new__(cls):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance

a = Singleton()
b = Singleton()

print('a is b?', a is b)

a is b? True


## Методы `__getattr__`, `__getattribute__`, `__setattr__`, `__delattr__`

In [6]:
class Researcher():

    # Метод вызывается при получении несуществующего атрибута
    def __getattr__(self, name):
        print('INFO: Доступ к несуществующему атрибуту "{}"'.format(name))

    # Метод вызывается при обращении к либому атрибуту, в том числе к несуществующему
    def __getattribute__(self, name):
        print('INFO: Доступ к атрибуту "{}"'.format(name))
        return object.__getattribute__(self, name)

    # Метод вызывается при присваивании значения атрибуту
    def __setattr__(self, name, value):
        print('INFO: Устанавливается значение "{}" атрибуту "{}"'.format(value, name))
        return object.__setattr__(self, name, value)

    # Метод вызывается при удалении атрибута
    def __delattr__(self, name):
        value = getattr(self, name)
        print('INFO: Удаляется атрибут "{}" со значением "{}"'.format(name, value))
        object.__delattr__(self, name)



obj = Researcher()

In [7]:
obj.attr # Доступ к несуществующему атрибуту

INFO: Доступ к атрибуту "attr"
INFO: Доступ к несуществующему атрибуту "attr"


In [8]:
obj.method() # Доступ к несуществующему методу

INFO: Доступ к атрибуту "method"
INFO: Доступ к несуществующему атрибуту "method"


TypeError: 'NoneType' object is not callable

In [9]:
obj.math = True # Установка значения атрибуту

INFO: Устанавливается значение "True" атрибуту "math"


In [10]:
obj.math # Доступ к атрибуту

INFO: Доступ к атрибуту "math"


True

In [11]:
del obj.math # Удаление атрибута

INFO: Доступ к атрибуту "math"
INFO: Удаляется атрибут "math" со значением "True"


## Метод `__call__`

In [12]:
class Logger():

    def __init__(self, filename):
        self.filename = filename

    def __call__(self, func):
        print('Logging to:', self.filename)
        return func


logger = Logger('log.txt')

# Далее в коде мы можем вызвать объект как функцию, которая будет что-то выполнять:
logger(user_function) # Скобки инициируют вызов метода __call__()


# Аналогично мы можем выполнить через декоратор в который передается функция user_function():
@logger
def user_function():
    pass

NameError: name 'user_function' is not defined

## Метод `__add__`

Перегрузка оператора сложения `+` или вызов магического метода `__add__()`:

In [13]:
import random

class NoisyInt():

    def __init__(self, value):
        self.value = value

    def __add__(self, obj):
        noise = random.uniform(-1, 1) # Подмешаем "соли" в результат
        return self.value + obj.value + noise


a = NoisyInt(10)
b = NoisyInt(20)

for _ in range(3):
    print(a + b) # Выводится результат сложения (a + b + случайное число)

29.728800594892032
30.814176869923205
29.528867906123434


Результат - разные числа, так как каждый раз подмешивается случайное значение в переопределенном методе `__add__()`

## Метод `__getitem__`, `__setitem__`

Метод определяет поведение объекта при доступе или установке значения по индексу или ключу: `obj[key]`.

Пример реализации доступа к элементам начиная с индекса 1, а не 0:

In [14]:
class PascalList():

    def __init__(self, original_list=None):
        self.container = original_list or []

    def __getitem__(self, key):
        return self.container[key - 1]

    def __setitem__(self, key, value):
        self.container[key - 1] = value


numbers = PascalList([1, 2, 3, 4, 5])

print(numbers[0])
print(numbers[1])
print(numbers[2])

5
1
2
